# 04. K-Nearest Neighbors (KNN) | خوارزمية الجيران الأقرب (KNN)

## 📚 Prerequisites (What You Need First) | المتطلبات الأساسية

**BEFORE starting this notebook**, you should have completed:
- ✅ **Unit 3, Examples 1-3**: Logistic Regression, Decision Trees, and SVM
- ✅ **Understanding of distance metrics**: Euclidean distance, Manhattan distance
- ✅ **Understanding of feature scaling**: Why scaling matters for distance-based algorithms

**If you haven't completed these**, you might struggle with:
- Understanding how KNN makes predictions based on neighbors
- Knowing why feature scaling is critical for KNN
- Understanding how to choose the right K value

---

## 🔗 Where This Notebook Fits | مكان هذا الدفتر

**This is Unit 3, Example 4** - it introduces instance-based classification!

**Why this example FOURTH in Unit 3?**
- **Before** you can use KNN, you need to understand basic classification
- **Before** you can choose K, you need to understand overfitting vs underfitting
- **Before** you can use distance metrics, you need to understand feature scaling

**Builds on**: 
- 📓 Unit 3, Example 1: Logistic Regression (classification basics)
- 📓 Unit 3, Example 2: Decision Trees (overfitting concepts)
- 📓 Unit 3, Example 3: SVM (feature scaling importance)

**Leads to**: 
- 📓 Unit 4: Clustering (K-Means uses similar distance concepts)
- 📓 Unit 5: Model Selection (hyperparameter tuning for K)
- 📓 All instance-based learning algorithms

**Why this order?**
1. KNN is **simple to understand** (just find nearest neighbors)
2. KNN demonstrates **lazy learning** (no training, just prediction)
3. KNN shows **distance-based classification** (different from other methods)

---

## The Story: Learning from Neighbors | القصة: التعلم من الجيران

Imagine you're moving to a new neighborhood. **Before** KNN, you make decisions alone. **After** KNN, you look at your neighbors: "What do most of my neighbors do? I'll do that too!" - simple and effective!

Same with machine learning: **Before** KNN, we train complex models. **After** KNN, we just find the nearest neighbors and predict based on them - simple but powerful!

---

## Why KNN Matters | لماذا يهم KNN؟

KNN is a simple but powerful classifier:
- **Simple**: Easy to understand and implement
- **No Training**: Lazy learning - no model training needed
- **Non-Parametric**: Makes no assumptions about data distribution
- **Versatile**: Works for classification and regression
- **Effective**: Often performs well on many datasets

## Learning Objectives | أهداف التعلم
1. Build KNN classifiers
2. Understand how KNN makes predictions (nearest neighbors)
3. Choose the right K value (avoid overfitting/underfitting)
4. Understand distance metrics (Euclidean, Manhattan)
5. Know why feature scaling is critical for KNN
6. Compare KNN with other classification algorithms
7. Understand when to use KNN vs other methods


In [ ]:
# Step 1: Import necessary libraries
# These libraries help us build KNN classification models

import pandas as pd  # For data manipulation
import numpy as np   # For numerical operations
import matplotlib.pyplot as plt  # For visualizations
import seaborn as sns  # For beautiful plots
from sklearn.model_selection import train_test_split  # For splitting data
from sklearn.neighbors import KNeighborsClassifier  # KNN classifier
from sklearn.preprocessing import StandardScaler  # CRITICAL for KNN! Must scale features
from sklearn.metrics import (
    accuracy_score,        # Classification accuracy
    classification_report,  # Comprehensive metrics
    confusion_matrix,      # Confusion matrix
    roc_auc_score,         # AUC score
    roc_curve              # ROC curve
)
from sklearn.datasets import load_breast_cancer  # Real-world dataset

print("✅ Libraries imported successfully!")
print("\n📚 Key KNN Concepts:")
print("   - KNeighborsClassifier: KNN classifier (finds K nearest neighbors)")
print("   - K parameter: Number of neighbors to consider (critical hyperparameter)")
print("   - Distance metrics: Euclidean (default), Manhattan, etc.")
print("   - Lazy learning: No training phase, just prediction")
print("\n   ⚠️  IMPORTANT: KNN requires feature scaling! Always use StandardScaler!")
print("   💡 Why? Distance calculations are affected by feature scales!")


## Part 1: Understanding KNN | الجزء الأول: فهم KNN

### 🔗 What is KNN? | ما هو KNN؟

**K-Nearest Neighbors (KNN)** is a simple, instance-based learning algorithm:

1. **No Training Phase**: KNN doesn't "train" a model - it stores all training data
2. **Prediction Phase**: For a new data point, KNN finds the K nearest neighbors
3. **Voting**: The class of the new point is determined by majority vote of its K neighbors

**Key Concepts:**
- **K**: Number of neighbors to consider (hyperparameter)
- **Distance Metric**: How to measure "nearest" (Euclidean, Manhattan, etc.)
- **Lazy Learning**: No model is built during training - all computation happens during prediction

**Why Feature Scaling Matters:**
- KNN uses **distance** to find neighbors
- If features have different scales (e.g., age: 0-100, income: 0-100000), distance will be dominated by the larger scale
- **Solution**: Always scale features before using KNN!

---

## Step 1: Load and Prepare Data | الخطوة 1: تحميل وإعداد البيانات

**BEFORE**: We need to understand the dataset structure.

**AFTER**: We'll load the Wisconsin Breast Cancer dataset and prepare it for KNN!

**Why this dataset**: 
- Real-world medical data
- Binary classification (perfect for KNN demonstration)
- Multiple features (shows importance of scaling)


In [ ]:
# Load the Wisconsin Breast Cancer dataset
# This is REAL medical data for binary classification

print("\n" + "=" * 60)
print("Loading Wisconsin Breast Cancer Dataset")
print("تحميل مجموعة بيانات سرطان الثدي من ويسكونسن")
print("=" * 60)

cancer_data = load_breast_cancer()

# Create DataFrame
df = pd.DataFrame(cancer_data.data, columns=cancer_data.feature_names)
df['target'] = cancer_data.target  # 0 = Malignant, 1 = Benign

print(f"\n✅ Dataset loaded!")
print(f"   📊 Shape: {df.shape}")
print(f"   🎯 Classes: {len(np.unique(df['target']))} (binary classification)")
print(f"   📈 Features: {len(df.columns)-1} features (measurements from cell nuclei)")
print(f"\n🔍 Class Distribution:")
print(df['target'].value_counts().to_string())
print(f"\n💡 This is REAL medical diagnosis data from University of Wisconsin")
print(f"   - Features are measurements from cell nuclei")
print(f"   - Target: 0 = Malignant (cancerous), 1 = Benign (non-cancerous)")

# Separate features and target
X = df.drop('target', axis=1)
y = df['target']

print(f"\n✅ Features and target separated!")
print(f"   X shape: {X.shape}")
print(f"   y shape: {y.shape}")


## Step 2: Understanding Why Feature Scaling is Critical | الخطوة 2: فهم لماذا تحجيم الميزات مهم جداً

**BEFORE**: KNN uses distance to find neighbors. If features have different scales, distance will be wrong!

**AFTER**: We'll demonstrate why scaling matters by comparing scaled vs unscaled KNN!

**Why this matters**: 
- **Unscaled**: Features with larger values dominate distance calculations
- **Scaled**: All features contribute equally to distance calculations
- **Result**: Scaling dramatically improves KNN performance!


In [ ]:
# Demonstrate why feature scaling is critical for KNN
# We'll compare KNN with and without scaling

print("\n" + "=" * 60)
print("Demonstrating Feature Scaling Importance")
print("إظهار أهمية تحجيم الميزات")
print("=" * 60)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=123, stratify=y
)

print(f"\n✅ Data split!")
print(f"   Training samples: {len(X_train)}")
print(f"   Test samples: {len(X_test)}")

# Show feature scales BEFORE scaling
print(f"\n📊 Feature Scales BEFORE Scaling:")
print(f"   Mean of first feature: {X_train.iloc[:, 0].mean():.2f}")
print(f"   Std of first feature: {X_train.iloc[:, 0].std():.2f}")
print(f"   Mean of last feature: {X_train.iloc[:, -1].mean():.2f}")
print(f"   Std of last feature: {X_train.iloc[:, -1].std():.2f}")
print(f"\n💡 Notice: Features have VERY different scales!")
print(f"   - Some features range from 0-1")
print(f"   - Others range from 0-1000+")
print(f"   - This will make distance calculations dominated by large-scale features!")

# Train KNN WITHOUT scaling (BAD!)
knn_unscaled = KNeighborsClassifier(n_neighbors=5)
knn_unscaled.fit(X_train, y_train)
y_pred_unscaled = knn_unscaled.predict(X_test)
acc_unscaled = accuracy_score(y_test, y_pred_unscaled)

print(f"\n❌ KNN WITHOUT Scaling:")
print(f"   Accuracy: {acc_unscaled:.4f} ({acc_unscaled*100:.2f}%)")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n✅ Features scaled using StandardScaler!")
print(f"   - Mean of all features: ~0")
print(f"   - Std of all features: ~1")
print(f"   - All features now contribute equally to distance!")

# Train KNN WITH scaling (GOOD!)
knn_scaled = KNeighborsClassifier(n_neighbors=5)
knn_scaled.fit(X_train_scaled, y_train)
y_pred_scaled = knn_scaled.predict(X_test_scaled)
acc_scaled = accuracy_score(y_test, y_pred_scaled)

print(f"\n✅ KNN WITH Scaling:")
print(f"   Accuracy: {acc_scaled:.4f} ({acc_scaled*100:.2f}%)")

improvement = acc_scaled - acc_unscaled
print(f"\n🎯 Improvement from Scaling: {improvement:.4f} ({improvement*100:.2f}%)")
print(f"\n💡 Key Learning Point:")
print(f"   - Feature scaling is CRITICAL for KNN!")
print(f"   - Without scaling: {acc_unscaled*100:.1f}% accuracy")
print(f"   - With scaling: {acc_scaled*100:.1f}% accuracy")
print(f"   - Always use StandardScaler before KNN!")


## Step 3: Choosing the Right K Value | الخطوة 3: اختيار قيمة K الصحيحة

**BEFORE**: We need to understand how K affects model performance.

**AFTER**: We'll test different K values and find the optimal one!

**Why K matters**: 
- **K too small (K=1)**: Overfitting - model is too sensitive to noise
- **K too large**: Underfitting - model loses local patterns
- **Optimal K**: Balances bias and variance for best generalization

**Rule of thumb**: Start with K = √n (square root of number of samples), then tune!


In [ ]:
# Find the optimal K value
# We'll test different K values and see which gives best performance

print("\n" + "=" * 60)
print("Finding Optimal K Value")
print("إيجاد قيمة K الأمثل")
print("=" * 60)

# Test different K values
k_values = range(1, 31, 2)  # Test odd numbers from 1 to 29
train_scores = []
test_scores = []

print(f"\n🔍 Testing K values from 1 to 29 (odd numbers only)...")
print(f"   Why odd? Avoids ties in binary classification!")

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    
    train_pred = knn.predict(X_train_scaled)
    test_pred = knn.predict(X_test_scaled)
    
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    
    train_scores.append(train_acc)
    test_scores.append(test_acc)

# Find best K
best_k_idx = np.argmax(test_scores)
best_k = k_values[best_k_idx]
best_test_acc = test_scores[best_k_idx]

print(f"\n✅ Best K value: {best_k}")
print(f"   Test Accuracy: {best_test_acc:.4f} ({best_test_acc*100:.2f}%)")

# Visualize K vs Accuracy
plt.figure(figsize=(12, 6))
plt.plot(k_values, train_scores, 'o-', label='Training Accuracy', linewidth=2, markersize=8)
plt.plot(k_values, test_scores, 's-', label='Test Accuracy', linewidth=2, markersize=8)
plt.axvline(x=best_k, color='r', linestyle='--', linewidth=2, label=f'Best K = {best_k}')
plt.xlabel('K Value (Number of Neighbors)', fontsize=12, fontweight='bold')
plt.ylabel('Accuracy', fontsize=12, fontweight='bold')
plt.title('KNN: Finding Optimal K Value | KNN: إيجاد قيمة K الأمثل', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n💡 Key Observations:")
print(f"   - K=1: High training accuracy, lower test accuracy (overfitting)")
print(f"   - K too large: Lower accuracy on both (underfitting)")
print(f"   - Optimal K={best_k}: Best balance (test accuracy: {best_test_acc*100:.2f}%)")
print(f"\n📚 Learning Points:")
print(f"   - Small K: Model is too sensitive to noise (overfitting)")
print(f"   - Large K: Model loses local patterns (underfitting)")
print(f"   - Optimal K: Balances bias and variance for best generalization")


## Step 4: Train Final KNN Model and Evaluate | الخطوة 4: تدريب نموذج KNN النهائي والتقييم

**BEFORE**: We found the optimal K value.

**AFTER**: We'll train the final model with optimal K and evaluate it comprehensively!

**Why comprehensive evaluation matters**: 
- Accuracy alone doesn't tell the full story
- Precision, Recall, F1, and Confusion Matrix give complete picture
- ROC Curve shows model's ability to separate classes


In [ ]:
# Train final KNN model with optimal K and evaluate comprehensively

print("\n" + "=" * 60)
print("Training Final KNN Model with Optimal K")
print("تدريب نموذج KNN النهائي بقيمة K الأمثل")
print("=" * 60)

# Train with best K
knn_final = KNeighborsClassifier(n_neighbors=best_k)
knn_final.fit(X_train_scaled, y_train)

# Make predictions
y_train_pred = knn_final.predict(X_train_scaled)
y_test_pred = knn_final.predict(X_test_scaled)
y_test_proba = knn_final.predict_proba(X_test_scaled)[:, 1]  # Probabilities for class 1

# Calculate metrics
from sklearn.metrics import precision_score, recall_score, f1_score

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_auc = roc_auc_score(y_test, y_test_proba)

print(f"\n📊 Final KNN Model Performance (K={best_k}):")
print(f"   Training Accuracy: {train_acc:.4f} ({train_acc*100:.2f}%)")
print(f"   Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"   Test Precision: {test_precision:.4f} ({test_precision*100:.2f}%)")
print(f"   Test Recall: {test_recall:.4f} ({test_recall*100:.2f}%)")
print(f"   Test F1-Score: {test_f1:.4f} ({test_f1*100:.2f}%)")
print(f"   Test AUC: {test_auc:.4f} ({test_auc*100:.2f}%)")

print(f"\n✅ KNN Model trained and evaluated successfully!")
print(f"\n💡 Interpretation:")
print(f"   - Accuracy: Overall correctness ({test_acc*100:.1f}% correct)")
print(f"   - Precision: Of predicted positives, {test_precision*100:.1f}% are actually positive")
print(f"   - Recall: Of actual positives, we caught {test_recall*100:.1f}%")
print(f"   - F1: Balance between precision and recall ({test_f1*100:.1f}%)")
print(f"   - AUC: Model's ability to separate classes ({test_auc*100:.1f}%)")

# Confusion Matrix
cm = confusion_matrix(y_test, y_test_pred)
print(f"\n📊 Confusion Matrix:")
print(cm)
print(f"\n   True Negatives (TN): {cm[0,0]} - Correctly predicted Malignant")
print(f"   False Positives (FP): {cm[0,1]} - Predicted Benign but actually Malignant")
print(f"   False Negatives (FN): {cm[1,0]} - Predicted Malignant but actually Benign")
print(f"   True Positives (TP): {cm[1,1]} - Correctly predicted Benign")

# Visualize Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar_kws={'label': 'Count'})
plt.xlabel('Predicted Label', fontsize=12, fontweight='bold')
plt.ylabel('Actual Label', fontsize=12, fontweight='bold')
plt.title('KNN Confusion Matrix | مصفوفة الارتباك لـ KNN', fontsize=14, fontweight='bold')
plt.xticks([0.5, 1.5], ['Malignant (0)', 'Benign (1)'])
plt.yticks([0.5, 1.5], ['Malignant (0)', 'Benign (1)'])
plt.tight_layout()
plt.show()

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_test_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, linewidth=2, label=f'KNN (AUC = {test_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
plt.xlabel('False Positive Rate', fontsize=12, fontweight='bold')
plt.ylabel('True Positive Rate', fontsize=12, fontweight='bold')
plt.title('KNN ROC Curve | منحنى ROC لـ KNN', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n✅ Comprehensive evaluation complete!")


## Step 5: When to Use KNN vs Other Algorithms | الخطوة 5: متى نستخدم KNN مقابل الخوارزميات الأخرى

**BEFORE**: We understand how KNN works.

**AFTER**: We'll understand when KNN is the right choice and when to use other algorithms!

**Decision Framework:**

| Scenario | Best Algorithm | Why |
|----------|---------------|-----|
| **Small dataset, interpretability important** | Decision Trees | Easy to understand |
| **Large dataset, need fast predictions** | Logistic Regression | Fast, efficient |
| **Non-linear patterns, need optimal margin** | SVM | Maximum margin boundaries |
| **No clear pattern, need simple solution** | KNN | Simple, no assumptions |
| **Need feature importance** | Random Forest | Built-in feature importance |
| **Lazy learning acceptable** | KNN | No training phase needed |

**KNN Advantages:**
- ✅ Simple to understand and implement
- ✅ No assumptions about data distribution
- ✅ Works well for non-linear patterns
- ✅ Can be used for both classification and regression

**KNN Disadvantages:**
- ❌ Slow prediction (must compute distances to all training points)
- ❌ Sensitive to irrelevant features
- ❌ Requires feature scaling
- ❌ Memory intensive (stores all training data)
- ❌ Sensitive to K value choice


## Summary | الملخص

### What We Learned | ما تعلمناه

1. **KNN Basics**: KNN finds K nearest neighbors and predicts based on majority vote
2. **Feature Scaling**: CRITICAL for KNN - always use StandardScaler!
3. **Choosing K**: Balance between overfitting (small K) and underfitting (large K)
4. **Lazy Learning**: KNN doesn't train a model - all computation happens during prediction
5. **Distance Metrics**: Euclidean distance (default) measures "nearest"
6. **When to Use**: Simple solution, no assumptions, non-linear patterns

### Key Takeaways | النقاط الرئيسية

- ⚠️ **Always scale features before KNN** - distance calculations depend on feature scales
- 🎯 **Choose K carefully** - too small = overfitting, too large = underfitting
- 💡 **KNN is simple but powerful** - often performs well with proper preprocessing
- 📊 **Evaluate comprehensively** - use accuracy, precision, recall, F1, AUC, confusion matrix
- 🔍 **Understand trade-offs** - KNN is slow for large datasets but simple to understand

### Next Steps | الخطوات التالية

- 📓 **Unit 4: Clustering** - K-Means uses similar distance concepts
- 📓 **Unit 5: Model Selection** - Hyperparameter tuning for K
- 📓 **Advanced Topics** - Weighted KNN, distance metrics, approximate nearest neighbors

---

**Congratulations!** 🎉 You've completed all classification algorithms in Unit 3!
**تهانينا!** 🎉 لقد أكملت جميع خوارزميات التصنيف في الوحدة 3!
a